In [1]:
from scipy.stats import norm
from scipy.stats import poisson
from sklearn.linear_model import LogisticRegression
import numpy as np

In [2]:
par_poisson = lambda mu: poisson(mu=mu)

In [3]:
dist = par_poisson(10)
print(dist)
p = dist.rvs(100)
print(np.mean(p))

dist = par_poisson(2)
p = dist.rvs(100)
print(np.mean(p))

9.77
1.92


In [5]:
model = LogisticRegression()
params ={'LogisticRegression': {'C': lambda loc : norm(loc=loc, scale=1),'penalty':['l2'],'solver':['lbfgs','newton-cg','sag']},
        'XGBClassifier': {'n_estimators': lambda mu: poisson(mu=mu), 'max_depth': lambda mu: poisson(mu=mu)}}

In [11]:
for p in params['LogisticRegression']:
    print(p)

C
penalty
solver


In [6]:
parameters = params[type(model).__name__]

In [9]:
model.penalty

'l2'

In [12]:
d = {'a':[1]}

In [13]:
d['b'] = [1]

In [14]:
d

{'a': [1], 'b': [1]}

In [15]:
model = LogisticRegression()

In [16]:
getattr(model,'C')

1.0

In [17]:
setattr(model,'C',2)

In [18]:
getattr(model,'C')

2

In [20]:
import random

In [21]:
x = [[1,2,3],[2,3,4],[3,4,5]]

In [24]:
y = random.sample(x,1)[0]

In [25]:
y

[1, 2, 3]

In [26]:
y = [0]

In [27]:
x

[[1, 2, 3], [2, 3, 4], [3, 4, 5]]

In [1]:
from copy import copy, deepcopy
from irace2 import irace, dummy_stats_test
import itertools
import numpy as np
from sampling_functions import norm_sample, truncated_poisson, truncated_skellam
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split,StratifiedShuffleSplit,cross_val_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import scipy.stats as ss
from scipy.stats import norm, poisson, skellam
from tqdm import tqdm
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

In [7]:
models = [LogisticRegression(C=1), 
    RandomForestClassifier(n_estimators=100,max_depth=5,ccp_alpha=0.0),
    SVC(C=1,coef0=0.0),
    XGBClassifier(n_estimators=100,max_depth=6,subsample=1)]

In [8]:
parameters_dict = {
    'LogisticRegression': {'C': lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
                            'penalty':['l2'],
                            'solver':['lbfgs','newton-cg','sag']},
    'SVC':{'C':lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
            'coef0': lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
            'kernel':['linear','poly','rbf','sigmoid'],
            'decision_function_shape':['ovo','ovr']},
    'RandomForestClassifier': {'n_estimators': lambda loc: truncated_skellam(loc, mu1=10, mu2=10, min=1), 
                                'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1),
                                'max_features':['sqrt', 'log2', None],
                                'ccp_alpha':lambda loc : norm_sample(loc=loc, scale=0.1, min= 1e-3)
                                },
    'XGBClassifier': {'tree_method': ['auto','exact','approx'], 
                        'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1),
                        'booster':['gbtree','dart'],
                        'subsample':lambda loc : norm_sample(loc=loc, scale=0.3, min= 1e-2,max=1)}
}

In [11]:
population = {}
for model in models:
    par_names = parameters_dict[type(model).__name__]
    par_val = {name: getattr(model, name) for name in par_names}
    population.update({type(model).__name__:dict(par_val)})

In [14]:
for p in population:
    print(population[p])

{'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
{'n_estimators': 100, 'max_depth': 5, 'max_features': 'sqrt', 'ccp_alpha': 0.0}
{'C': 1, 'coef0': 0.0, 'kernel': 'rbf', 'decision_function_shape': 'ovr'}
{'tree_method': None, 'max_depth': 6, 'booster': None, 'subsample': 1}


In [15]:
print(population)

{'LogisticRegression': {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}, 'RandomForestClassifier': {'n_estimators': 100, 'max_depth': 5, 'max_features': 'sqrt', 'ccp_alpha': 0.0}, 'SVC': {'C': 1, 'coef0': 0.0, 'kernel': 'rbf', 'decision_function_shape': 'ovr'}, 'XGBClassifier': {'tree_method': None, 'max_depth': 6, 'booster': None, 'subsample': 1}}
